In [30]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals.joblib import Parallel, delayed
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import matthews_corrcoef
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from pandas import Series,DataFrame
import copy

score_col = ['SIFT_score','Polyphen2_HDIV_score','Polyphen2_HVAR_score','LRT_score_new','MutationTaster_score_new','MutationAssessor_score','FATHMM_score','GERP++_RS',\
'PROVEAN_score','VEST3_score','MetaSVM_score','MetaLR_score','M-CAP_score','REVEL_score','MutPred_score','CADD_phred','DANN_score','fathmm-MKL_coding_score','Eigen-raw',\
'Eigen-PC-raw','GenoCanyon_score','SiPhy_29way_logOdds','integrated_fitCons_score','GM12878_fitCons_score','H1-hESC_fitCons_score','HUVEC_fitCons_score',\
'phyloP100way_vertebrate','phyloP20way_mammalian','phastCons100way_vertebrate','phastCons20way_mammalian']
#print(len(score_col))

tar = 0.5

def feature_select(base,imp,best_mcc,imp_real):
    global tar
    if imp:
        for i in imp:
            basetemp = copy.deepcopy(base)
            basetemp.append(i)
            temp = copy.deepcopy(imp)
            #temp.remove(i)
            

            mcc = score(basetemp,temp)
            
            list1 = []
            list2 = []
            for j in basetemp:
                list1.append(imp_real.index(j))

            #print(list1)
            if mcc>0.8:
                for j in basetemp:
                    ff.write(str(j))
                    ff.write(',')
                    
                ff.write('\n')
                ff.write(str(mcc))
                ff.write('\n')
                
                
        
            if mcc>best_mcc:
                temp = temp[temp.index(i)+1:]
                if mcc > tar:
                    tar = mcc
                    print(list1)
                    print('now = ',mcc,'best= ',best_mcc)
                '''
                for n in base:
                    ff.write(str(n))
                    ff.write(',')
                ff.write(':')
                ff.write(str(mcc))
                ff.write('\n')  
                '''
                feature_select(basetemp,temp,mcc,imp_real)
def backward(base):
    global tar
    exist = 0
    index = -1
    best = 0
    if base:
        
        basetemp = copy.deepcopy(base)
        best = score(basetemp)
        for i in range(len(basetemp)):
            #print(i)
            dummy = []
            for j in range(len(basetemp)):
                if j !=i:
                    dummy.append(basetemp[j])
            score_with_out_i = score(dummy)
            print(dummy)
            print(score_with_out_i)
            if score_with_out_i > best:
                best = score_with_out_i
                exist = 1
                index = i
    if exist:
        return index,best
    else:
        return -1,best

def score_for_func(base):
    set1 = pd.read_csv('have_CHASMplus,FATHMMcancer_fathmm_training_set_real.csv')
    set2 = pd.read_csv('/data/wangs/MutaAnalysis/three.data0223/output/findal/functional_have_all_score_del_chasmplus_fathmmcancer_training_set.tsv', sep='\t')
    tr_set = DataFrame(set1)
    te_set = DataFrame(set2)
    #ss = ShuffleSplit(n_splits=1, random_state=0, test_size=0.10)
    train_set_label = tr_set['label']

    train_set_X = z_score(tr_set[base])[base]
    test_set_label = te_set['label']
    test_set_X = z_score(te_set[base])[base]

    return knn_work(train_set_label,train_set_X,test_set_label,test_set_X,base)  
    
def score(base):
    t_set = pd.read_csv('fathmm_train_new.csv')
    t_set = DataFrame(t_set)
    ss = ShuffleSplit(n_splits=1, random_state=0, test_size=0.10)
    for train_index, test_index in ss.split(t_set):
        train_set = t_set.iloc[train_index]
        test_set = t_set.iloc[test_index]
        
        train_set_label = train_set['label']
        
        train_set_X = z_score(train_set[base])[base]
        test_set_label = test_set['label']
        test_set_X = z_score(test_set[base])[base]

    return knn_work(train_set_label,train_set_X,test_set_label,test_set_X,base)  
    
    
def rf(train_set_label,train_set_X,test_set_label,test_set_X,base):

    rf = RandomForestClassifier(n_estimators = 500,n_jobs=-1)
    rf_X = rf.fit(train_set_X,train_set_label)
    #print(rf_X)
    #print(rf.feature_importances_)
    #name = base
    #importence = rf.feature_importances_
    r = rf.predict(test_set_X)
    rr= rf.predict_proba(test_set_X)
    #print(rr)
    score_pred = []
    for pred in rr:
        score_pred.append(pred[1])
    #FATHMM_Cancer_score = test_set.loc[:,'FATHMM_Cancer_score'] 
    #print(M_CAP_score)
    #CHASMplus = test_set.loc[:,'CHASMplus']
    #mcc = matthews_corrcoef(test_set_label, r)
    #print(matthews_corrcoef(test_set_label, r))
    #acc = accuracy_score(test_set_label, r)
    #print(accuracy_score(test_set_label, r))   
    auc = roc(test_set_label,score_pred)
    return auc
    
    
    
    #for i in range(30):
        #print(i,"     ",name[indices[i]],"     ",importence[indices[i]])
        #reuslt.write(name[indices[i]])
        #reuslt.write('  ')
        #reuslt.write(str(importence[indices[i]]))
        #reuslt.write('\n')


def knn_work(train_set_label,train_set_X,test_set_label,test_set_X,base):
    knn=KNeighborsClassifier(n_neighbors=799)
    '''
    train_set_label = train_set['label']
    train_set_X = train_set[best_list]
    test_set_label = test_set['label']
    test_set_X = test_set[best_list ]
    '''
    knn_X = knn.fit(train_set_X,train_set_label)
    r = knn.predict(test_set_X)
    rr= knn.predict_proba(test_set_X)
    #print(rr)
    score_pred = []
    for pred in rr:
        score_pred.append(pred[1])
    #FATHMM_Cancer_score = test_set.loc[:,'FATHMM_Cancer_score'] 
    #print(M_CAP_score)
    #CHASMplus = test_set.loc[:,'CHASMplus']
    #mcc = matthews_corrcoef(test_set_label, r)
    #print(matthews_corrcoef(test_set_label, r))
    #acc = accuracy_score(test_set_label, r)
    #print(accuracy_score(test_set_label, r))   
    auc = roc(test_set_label,score_pred)
    return auc

def roc(label,score3):

    fpr3, tpr3, _ = roc_curve(label, score3)
    roc_auc3 = auc(fpr3, tpr3)
    
    return  roc_auc3
    '''
    plt.figure()
    plt.plot(fpr1, tpr1, color='navy', lw=2, label='FATHMM_Cancer_score ROC curve (area = %0.2f)' % roc_auc1)
    plt.plot(fpr2, tpr2, color='yellow', lw=2, label='CHASMplus ROC curve (area = %0.2f)' % roc_auc2)
    plt.plot(fpr3, tpr3, color='red', lw=2, label='new score ROC curve (area = %0.2f)' % roc_auc3)
    #plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.01, 1])
    plt.ylim([0, 1.01])
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate')
    plt.title('FATHMM_Cancer_score vs our new_score')
    plt.legend(loc="lower right")
    plt.show()    
    '''   
    
def rf_work(train_set, test_set):
    score_col = ['SIFT_score','Polyphen2_HDIV_score','Polyphen2_HVAR_score','LRT_score_new','MutationTaster_score_new','MutationAssessor_score','FATHMM_score','GERP++_RS',\
    'PROVEAN_score','VEST3_score','MetaSVM_score','MetaLR_score','M-CAP_score','REVEL_score','MutPred_score','CADD_phred','DANN_score','fathmm-MKL_coding_score','Eigen-raw',\
    'Eigen-PC-raw','GenoCanyon_score','SiPhy_29way_logOdds','integrated_fitCons_score','GM12878_fitCons_score','H1-hESC_fitCons_score','HUVEC_fitCons_score',\
    'phyloP100way_vertebrate','phyloP20way_mammalian','phastCons100way_vertebrate','phastCons20way_mammalian']
    
    train_set_label = train_set['label']
    train_set_X = train_set[score_col ]
    test_set_label = test_set['label']
    test_set_X = test_set[score_col ]
     
    
    
    rf = RandomForestClassifier(n_estimators = 1000,verbose=True,n_jobs=-1)
    rf_X = rf.fit(train_set_X,train_set_label)
    print(rf_X)
    print(rf.feature_importances_)
    name = score_col
    importence = rf.feature_importances_
    r = rf.predict(test_set_X)
    acc = matthews_corrcoef(test_set_label, r)
    print(matthews_corrcoef(test_set_label, r))
    
    indices =np.argsort(importence)[::-1]
    
    reuslt = open('feature_importance.txt','w')
    reuslt.write('score_name')
    reuslt.write('  ')
    reuslt.write('feature_importances')
    reuslt.write('\n')
    
    
    
    for i in range(30):
        print(i,"     ",name[indices[i]],"     ",importence[indices[i]])
        reuslt.write(name[indices[i]])
        reuslt.write('  ')
        reuslt.write(str(importence[indices[i]]))
        reuslt.write('\n')
    
    return acc
def z_score(X):
    z_list = []
    z_list = pd.Series(z_list)
    for i in range(X.shape[1]):
        a = X.iloc[:, [i]]
        a_mean = a.mean(axis=0)  
        a_std = a.std(axis=0)  
        a1 = (a-a_mean)/a_std
        z_list = pd.concat([z_list,a1],axis=1)
        #print(z_list)
    #z_list = z_list.iloc[:,[1,z_list.shape[1]-1]]
    return z_list

def test(base,imp_test,b):
    return [],1
f = open('feature_importance.txt','r')
l = f.readline()
l = f.readline()
imp = []
while l:
    a = l.split('  ')
    imp.append(a[0])
    l = f.readline()
ff = open('result_auc_knn_799.txt','a')

index = 55
imp =  ['integrated_fitCons_score','SIFT_score','Polyphen2_HDIV_score','Polyphen2_HVAR_score',\
         'LRT_score_new','MutationAssessor_score','PROVEAN_score','VEST3_score','MetaSVM_score',\
         'MetaLR_score','M-CAP_score','REVEL_score','MutPred_score','CADD_phred','DANN_score',\
         'fathmm-MKL_coding_score','Eigen-raw','GERP++_RS','phyloP100way_vertebrate',\
         'phastCons100way_vertebrate','SiPhy_29way_logOdds','CONDEL','FATHMM_score','GenoCanyon_score','MutationTaster_score_new']
imp = ['MutationAssessor_score', 'PROVEAN_score', 'VEST3_score', 'MetaSVM_score',\
                   'M-CAP_score', 'REVEL_score', 'MutPred_score', 'CADD_phred', 'GERP++_RS', \
                   'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score']
imp_list = []
imp = ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'CADD_phred', 'GERP++_RS', 'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score']
imp_list.append(imp)
imp =['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score']
imp_list.append(imp)
imp =['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score']
imp_list.append(imp)
imp =['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score']
imp_list.append(imp)
imp = ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score']
imp_list.append(imp)
imp =['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score']
imp_list.append(imp)
imp =['MutationAssessor_score', 'VEST3_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score']
imp_list.append(imp)
imp =['VEST3_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score']
imp_list.append(imp)
imp =['MutPred_score', 'GERP++_RS', 'FATHMM_score']
imp_list.append(imp)
imp =['MutPred_score', 'FATHMM_score']
imp_list.append(imp)
print(imp_list)

score_list = []

for imp in imp_list:
    now = score(imp)
    print(now)
    score_list.append(now)
print(score_list)
print(imp_list)
'''
    print("#######################backward begin###################")

    while index != -1:
        index,best = backward(imp)
        if index != -1:
            imp.pop(index)
            print(imp)
            print("auc = ",best)

        else:
            print("#######################backward end###################")
            print(imp)
            print("auc = ",best)
'''

[['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'CADD_phred', 'GERP++_RS', 'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score'], ['VEST3_score', 'Mut

'\n    print("#######################backward begin###################")\n\n    while index != -1:\n        index,best = backward(imp)\n        if index != -1:\n            imp.pop(index)\n            print(imp)\n            print("auc = ",best)\n\n        else:\n            print("#######################backward end###################")\n            print(imp)\n            print("auc = ",best)\n'

In [31]:
####data temp 
score_list = [0.7602318872876739, 0.7592643490318427, 0.759700762640519, 0.7592018273659189, 0.7581179121477739, 0.7563654484227208, 0.75291623334819, 0.7493803545783192, 0.7427512009112068, 0.7326096914772446]
imp_list = [['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'CADD_phred', 'GERP++_RS', 'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'CONDEL', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'REVEL_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'M-CAP_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'MutPred_score', 'GERP++_RS', 'SiPhy_29way_logOdds', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MetaSVM_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score'], ['MutationAssessor_score', 'VEST3_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score'], ['VEST3_score', 'MutPred_score', 'GERP++_RS', 'FATHMM_score'], ['MutPred_score', 'GERP++_RS', 'FATHMM_score'], ['MutPred_score', 'FATHMM_score']]
a = 0



